In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv("../Data/all-data/house-votes-84-yeas.csv.bz2",sep="\t")
df.head()

,party,yea1,yea2,yea3,yea4,yea5,yea6,yea7,yea8,yea9,yea10,yea11,yea12,yea13,yea14,yea15,yea16
0,republican,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,1
1,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,0
2,democrat,0,1,1,0,1,1,0,0,0,0,1,0,1,1,0,0
3,democrat,0,1,1,0,0,1,0,0,0,0,1,0,1,0,0,1
4,democrat,1,1,1,0,1,1,0,0,0,0,1,0,1,1,1,1


### 1.1 Compute Priors

In [14]:
#1.1.1 Pr(party = D) Pr(party = R)

PD = sum(df.party == "democrat")/len(df.party)
PR = sum(df.party == "republican")/len(df.party)
print("Pr(party = D) = ",PD)
print("Pr(party = R) = ",PR)

Pr(party = D) =  0.6137931034482759
Pr(party = R) =  0.38620689655172413


In [15]:
#1.1.2
x8 = df["yea8"]
x8

0      0
1      0
2      0
3      0
4      0
      ..
430    0
431    1
432    0
433    0
434    0
Name: yea8, Length: 435, dtype: int64

In [16]:
#1.1.3 Pr(vote = 1)
PY = sum(x8 == 1)/len(x8)
PY

0.5563218390804597

In [17]:
#1.1.3 Pr(vote = 0)
PN = sum(x8 == 0)/len(x8)
PN

0.4436781609195402

In [18]:
#1.1.4 Pr(vote = Y|party = D)
df1 = df.groupby(['party']).mean()
df1

,yea1,yea2,yea3,yea4,yea5,yea6,yea7,yea8,yea9,yea10,yea11,yea12,yea13,yea14,yea15,yea16
party,,,,,,,,,,,,,,,,
democrat,0.584270,0.449438,0.865169,0.052434,0.205993,0.460674,0.749064,0.816479,0.704120,0.464419,0.483146,0.134831,0.273408,0.337079,0.599251,0.647940
republican,0.184524,0.446429,0.130952,0.970238,0.934524,0.886905,0.232143,0.142857,0.113095,0.547619,0.125000,0.803571,0.809524,0.940476,0.083333,0.571429


In [19]:
P_8Y_D = np.mean(df.yea8[df.party == 'democrat'])
P_8N_D = 1 - P_8Y_D
P_8Y_R = np.mean(df.yea8[df.party == 'republican'])
P_8N_R = 1 - P_8Y_R
print("Pr(vote = Y|party = D)", P_8Y_D)
print("Pr(vote = N|party = D)", P_8N_D)
print("Pr(vote = Y|party = R)", P_8Y_R)
print("Pr(vote = N|party = R)", P_8N_R)

Pr(vote = Y|party = D) 0.8164794007490637
Pr(vote = N|party = D) 0.18352059925093633
Pr(vote = Y|party = R) 0.14285714285714285
Pr(vote = N|party = R) 0.8571428571428572


### 1.2 Predict using Bayes theorem

In [23]:
# if yes, P_D_8Y = P_8Y_D * PD / P8Y
# if no, P_D_8N = P_8N_D * PD / P8N
P_D_8 = np.where(df.yea8, P_8Y_D * PD / PY, P_8N_D * PD / PN)

In [26]:
democrat = P_D_8 > 0.5

In [28]:
#1.2.3
np.mean(democrat == (df.party == "democrat"))

0.832183908045977

In [32]:
#1.2.3
from sklearn.metrics import confusion_matrix
print("Confusion Matrix:\n", confusion_matrix(df.party == 'democrat', democrat))

Confusion Matrix:
 [[144  24]
 [ 49 218]]
